In [74]:
from aux_fun import get_tokens_inst_iter_folders
import pandas as pd
import numpy as np
from tqdm import tqdm

In [5]:
# Select the instruments to keep by setting the value to True
instrument_to_keep = {
        "distorted0": False,
        "distorted1": False,
        "distorted2": False,
        "clean0": False,
        "clean1": False,
        "bass": True,
        "leads": False,    
        "pads": False,
        "drums": False,
    }

path_to_general_read_folder = "..\data\DadaGP\DadaGP-v1.1"
path_to_general_write_folder = "..\data\BGTG\BGTG-v0.1"

# get_tokens_inst_iter_folders(instrument_to_keep, path_to_general_read_folder, path_to_general_write_folder)

# 8min10s to generate all the token files with only bass and clean0

# 5min30s to generate all the bass tokens without checking if there is bass or not
# 4min49s for bass tokens if we check in the whole file if there is bass or not
# 6min37s to generate the bass tokens when checking at track-level the presence of bass (11 230 tracks without bass)

# 8min12s to generate the token files with the guitars (clean and distorted)

In [25]:
# Take a look at the csv provided by Alexandre

df_rg = pd.read_csv('..\\data\\function_guitar-ext-with-predictions.csv')

In [7]:
# We take a look at a specific track

track_sample = np.random.choice(df_rg['Track_Name'].unique())

df_sample = df_rg[df_rg['Track_Name'] == track_sample]

df_sample.shape

(290, 40)

In [87]:
track_sample = 'Arrows Fly\nEdguy' # Good example

df_sample = df_rg[df_rg['Track_Name'] == track_sample]

display("Instruments in the track:", df_sample['Instrument'].unique(),
        "Number of parts:", df_sample['Partie'].unique())

# We need to link the part to the instrument

df_inst_part = df_sample[['Partie', 'Instrument']].drop_duplicates()

display(df_inst_part)

# Let's hope the order in DadaGP is the same as in the csv


'Instruments in the track:'

array(['bass', 'distorted', 'clean'], dtype=object)

'Number of parts:'

array(['Bass', 'Guitar 1', 'Guitar 2', 'Guitar 3', 'Track 10', 'Track 2'],
      dtype=object)

,Partie,Instrument
1191110,Bass,bass
1191285,Guitar 1,distorted
1191466,Guitar 2,distorted
1191635,Guitar 3,distorted
1191659,Track 10,clean
1191662,Track 2,clean


In [88]:

# Add column DadaGP_name that adds a number to the distorted and clean instruments

df_inst_part['DadaGP_name'] = df_inst_part['Instrument']

def add_number(inst_list):
    nb_clean = 0
    nb_distorted = 0
    dadagp_inst = []
    for x in inst_list:
        if x == 'clean':
            dadagp_inst.append(x + str(nb_clean))
            nb_clean += 1
        elif x == 'distorted':
            dadagp_inst.append(x + str(nb_distorted))
            nb_distorted += 1
        else:
            dadagp_inst.append(x)
    return dadagp_inst

df_inst_part['DadaGP_name'] = add_number(df_inst_part['Instrument'])

display(df_inst_part)
# It seems to work


,Partie,Instrument,DadaGP_name
1191110,Bass,bass,bass
1191285,Guitar 1,distorted,distorted0
1191466,Guitar 2,distorted,distorted1
1191635,Guitar 3,distorted,distorted2
1191659,Track 10,clean,clean0
1191662,Track 2,clean,clean1


In [ ]:
df_inst_part = df_rg[['Track_Name', 'Partie', 'Instrument']]
df_inst_part = df_inst_part.drop_duplicates(subset=['Track_Name', 'Partie'])
total_df_track = pd.DataFrame()

# For each track we add the DadaGP_name column
for track in tqdm(df_inst_part['Track_Name'].unique()):
    df_track = df_inst_part[df_inst_part['Track_Name'] == track].copy() # We copy to avoid SettingWithCopyWarning
    df_track.loc[:, 'DadaGP_name'] = add_number(df_track['Instrument'])
    
    # Join df_track with df_inst_part
    total_df_track = pd.concat([total_df_track, df_track])

df_inst_part.shape, total_df_track.shape

100%|██████████| 21859/21859 [01:30<00:00, 241.00it/s]


((53832, 3), (53832, 4))

In [ ]:
# Perform a join on Track_Name and Partie to add the column DadaGP_name to the original dataframe
df_rg_augmented = df_rg.merge(total_df_track[['Track_Name', 'Partie', 'DadaGP_name']], on=['Track_Name', 'Partie'], how='left')

print(df_rg.shape, df_rg_augmented.shape)

(4165789, 40) (4165789, 41)


In [86]:
# now let's take a look at the instruments that generally are rythmic
df_rg_augmented.columns
# We remove the features columns of the dataset

df_rg_aug_essentials = df_rg_augmented[['Fichier', 'Track_Name', 'Partie', 'Instrument', 'DadaGP_name', 'Mesure', 'rg-estimation']]

df_rg_aug_essentials.head()

,Fichier,Track_Name,Partie,Instrument,DadaGP_name,Mesure,rg-estimation
0,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,distorted0,1,2.570728e-07
1,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,distorted0,2,4.580104e-07
2,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,distorted0,3,6.601997e-06
3,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,distorted0,4,4.111985e-04
4,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,distorted0,5,4.352387e-05


In [ ]:
# Now let's loop again on the tracks and take a look at what instruments are generally rythmic

# First describe column rg-estimation
df_rg_aug_essentials['rg-estimation'].describe()

# We could loop on tracks, within tracks loop on measures and for each measure take the instrument that has the highest rg-estimation
# But that would be very long?
# Maybe we can just say if rg-estimation > thr then the instrument is rythmic

thr = 0.5 # General decision threshold in the paper that is used to determine if an instrument is rythmic or not
df_rg_aug_essentials.loc[:, 'is_rythmic'] = df_rg_aug_essentials['rg-estimation'] > thr

display(df_rg_aug_essentials['is_rythmic'].value_counts())


is_rythmic
False    3494864
True      670925
Name: count, dtype: int64

In [ ]:
# However, now that I need this I need to build a dataframe with for each track and each instrument its proportion of rythmic measures

total_df_rythmic = pd.DataFrame()
i=0
# For each track we add the DadaGP_name column
for track in tqdm(df_rg_aug_essentials['Track_Name'].unique()):
    i=i+1
    df_rythmic = df_rg_aug_essentials[df_rg_aug_essentials['Track_Name'] == track].copy() # We copy to avoid SettingWithCopyWarning
    for instrument in df_rythmic['DadaGP_name'].unique():
        df_inst = df_rythmic[df_rythmic['DadaGP_name'] == instrument]
        prop_rythmic = df_inst['is_rythmic'].mean()
        df_rythmic.loc[df_rythmic['DadaGP_name'] == instrument, 'prop_rythmic'] = prop_rythmic  
    
    # The instrument with the highest proportion of rythmic measures is the rythmic instrument
    if df_rythmic['prop_rythmic'].isnull().all(): # No rythmic instrument at all in the track
        df_rythmic.loc[:, 'is_track_rythmic'] = False
    
    else:
        # Add column is_track_rythmic set to 1 at the rows where prop_rythmic is its max
        df_rythmic.loc[:, 'is_track_rythmic'] = df_rythmic['prop_rythmic'] == df_rythmic['prop_rythmic'].max()
          
    # Concatenate the dataframe
    total_df_rythmic = pd.concat([total_df_rythmic, df_rythmic])
    
    if i==100: # Test on 100 tracks
        break

# Around an hour of computation according to tqdm...

  0%|          | 99/21859 [00:15<58:15,  6.22it/s]  


In [115]:
total_df_rythmic[total_df_rythmic['Track_Name'] == 'Isolated\nDr. Sin']

,Fichier,Track_Name,Partie,Instrument,DadaGP_name,Mesure,rg-estimation,is_rythmic,prop_rythmic,is_track_rythmic
0,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,distorted0,1,2.570728e-07,False,0.519084,False
1,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,distorted0,2,4.580104e-07,False,0.519084,False
2,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,distorted0,3,6.601997e-06,False,0.519084,False
3,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,distorted0,4,4.111985e-04,False,0.519084,False
4,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,distorted0,5,4.352387e-05,False,0.519084,False
...,...,...,...,...,...,...,...,...,...,...
1142920,../data/DadaGP8-gpif/Dr Sin - Isolated.gpif,Isolated\nDr. Sin,Trilha 1,distorted,distorted1,26,9.999995e-01,True,1.000000,True
1142921,../data/DadaGP8-gpif/Dr Sin - Isolated.gpif,Isolated\nDr. Sin,Trilha 1,distorted,distorted1,27,9.999988e-01,True,1.000000,True
1142922,../data/DadaGP8-gpif/Dr Sin - Isolated.gpif,Isolated\nDr. Sin,Trilha 1,distorted,distorted1,28,9.999994e-01,True,1.000000,True
1142923,../data/DadaGP8-gpif/Dr Sin - Isolated.gpif,Isolated\nDr. Sin,Trilha 1,distorted,distorted1,29,9.999986e-01,True,1.000000,True
